## Manuel Romero García
## Máster en Big Data y Data Science
## Práctica Visualización de datos

## 1. Carga de librerías y datos

### dataset elegido: Contratos de empresas de trabajo temporal según sector, sacados de la Clasificación nacional de Actividades (CNAE) del Instituto Nacional de Estadística (INE)

In [8]:
# -*- coding: utf-8 -*-
"""
Created on Sat May 13 20:02:25 2023

@author: mromg
"""

import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from plotnine import *
import ipywidgets as widgets
from IPython.display import display
import os

user = os.getenv("USERNAME")

rows_to_skip = [0, 1, 2, 3]

# mantener en descargas
df = pd.read_excel(f'C:/Users/{user}/Downloads/Trabajo_Temporal_CNAEs_2018_2022.xlsx', skiprows=rows_to_skip)

In [13]:
df

,"Agricultura, ganadería, caza y servicios relacionados con las mismas",Silvicultura y explotación forestal,Pesca y acuicultura,"Extracción de antracita, hulla y lignito",Extracción de crudo de petróleo y gas natural,Extracción de minerales metálicos,Otras industrias extractivas,Actividades de apoyo a las industrias extractivas,Industria de la alimentación,Fabricación de bebidas,...,"Actividades de bibliotecas, archivos, museos y otras actividades culturales",Actividades de juegos de azar y apuestas,"Actividades deportivas, recreativas y de entretenimiento",Actividades asociativas,"Reparación de ordenadores, efectos personales y artículos de uso doméstico",Otros servicios personales,Actividades de los hogares como empleadores de personal doméstico,Actividades de los hogares como productores de bienes y servicios para uso propio,Actividades de organizaciones y organismos extraterritoriales,No Clasificables
fecha,,,,,,,,,,,,,,,,,,,,,
2009-01-01,17880.0,8.0,63.0,2.0,2.0,0.0,22.0,5.0,12807.0,1116.0,...,111.0,269.0,1453.0,560.0,147.0,361.0,33.0,0.0,5.0,142.0
2009-02-01,12999.0,1.0,32.0,0.0,0.0,0.0,6.0,0.0,10336.0,779.0,...,188.0,285.0,1027.0,489.0,106.0,290.0,11.0,0.0,5.0,71.0
2009-03-01,10533.0,9.0,60.0,1.0,0.0,0.0,4.0,0.0,11246.0,934.0,...,4.0,82.0,1451.0,652.0,84.0,359.0,26.0,0.0,2.0,84.0
2009-04-01,15965.0,0.0,72.0,0.0,0.0,0.0,13.0,1.0,12883.0,1177.0,...,126.0,559.0,1404.0,530.0,76.0,268.0,20.0,0.0,6.0,99.0
2009-05-01,19964.0,0.0,77.0,0.0,0.0,0.0,12.0,0.0,11926.0,1216.0,...,256.0,398.0,2224.0,650.0,81.0,368.0,23.0,0.0,7.0,78.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-01,31189.0,8.0,9.0,1.0,0.0,9.0,11.0,1.0,34592.0,3514.0,...,128.0,395.0,1131.0,161.0,227.0,589.0,59.0,0.0,5.0,0.0
2022-09-01,35579.0,27.0,29.0,0.0,0.0,12.0,8.0,0.0,33569.0,4300.0,...,206.0,277.0,1802.0,169.0,188.0,549.0,34.0,0.0,7.0,0.0
2022-10-01,56436.0,18.0,27.0,0.0,0.0,13.0,18.0,0.0,34873.0,2529.0,...,250.0,209.0,2743.0,194.0,206.0,582.0,21.0,0.0,5.0,0.0


## 2. Recodificación de datos

In [9]:
# miramos las columnas
df.columns

# eliminamos el total
df.drop(columns=['TOTAL'], inplace=True)

# renombramos esto por lo que vamos a hacer abajo
df.rename(columns={"Unnamed: 0": "aaaaAño", "Unnamed: 1": "aaaaMes"}, inplace=True)

# eliminamos primera parte con número de las columnas (más fácil y rápido que usar regex)
df.columns = [col[4:] for col in df.columns]


# diccionario de meses
month_dict = {
    'Enero': '01',
    'Febrero': '02',
    'Marzo': '03',
    'Abril': '04',
    'Mayo': '05',
    'Junio': '06',
    'Julio': '07',
    'Agosto': '08',
    'Septiembre': '09',
    'Octubre': '10',
    'Noviembre': '11',
    'Diciembre': '12'
}

# Aplicamos diccionario
df['Mes'] = df['Mes'].map(month_dict)

df['Mes']

# eliminamos nas (filas de separación)
df.dropna(subset=['Mes'], inplace=True)

# construimos fecha: ffill para que todos los na tomen el valor anterior, añadimos separación por barra y añadimos el día 1
df['fecha'] = pd.to_datetime(df['Año'].fillna(method='ffill').astype(int).astype(str) + '-' + df['Mes'].fillna(method='ffill') + '-01', format='%Y-%m-%d')

# fecha como índice
df.set_index('fecha', inplace=True)

# Eliminamos columna de año y mes
df.drop(['Año', 'Mes'], axis=1, inplace=True)

# tomamos los años desde la pandemia

four_years_ago = datetime.now() - timedelta(days=365*4)
## Lo de abajo, por si se ejecuta otro año que no sea 2023
## four_years_ago = datetime.datetime(2019, 5, 13, 16, 55, 30, 209464)
ts_df = df.loc[df.index >= four_years_ago]

# dropeo esto porque no conozco el contexto
ts_df.drop(columns=['Actividades relacionadas con el empleo'], inplace = True)

# cogemos la primera fila
first_row = ts_df.iloc[0]

# Sólo tomamos aquellos sectores dónde se firman más de 50 contratos (el resto puede tener un "efecto ancla": es más fácil crecer más cuándo es más pequeño)
ts_df = ts_df.loc[:, first_row > 50]

# cogemos la última fila para calcular el cambio
last_row = ts_df.iloc[-1]
pct_change = ((last_row - first_row) / first_row) * 100

## seleccionamos las cinco actividades que más evolución han presentado (problema: las actividades que registran perdidos en estas fechas no son tomadas en cuenta)
largest_var_columns = pct_change.abs().sort_values(ascending=False).head(5).index.tolist()

# creamos un df con las columnas elegidas
ts_df = ts_df[largest_var_columns]

ts_df.columns

# renombramos esta columna para mejor visualización de títulos
ts_df.rename(columns={'Actividades de agencias de viajes, operadores turísticos, servicios de reservas y actividades relacionadas con los mismos':
'Actividades de turismo y viajes'}, inplace=True)
    
# lo ponemos en formato largo (quitamos fechas del índice y las guardamos como columna)
ts_df_long = ts_df.reset_index().melt(id_vars='fecha', var_name='variable', value_name='value')

C:\Users\mromg\AppData\Local\Temp\ipykernel_3288\3371763815.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
ts_df_long

,fecha,variable,value
0,2019-06-01,Actividades de turismo y viajes,87.0
1,2019-07-01,Actividades de turismo y viajes,82.0
2,2019-08-01,Actividades de turismo y viajes,91.0
3,2019-09-01,Actividades de turismo y viajes,130.0
4,2019-10-01,Actividades de turismo y viajes,114.0
...,...,...,...
210,2022-08-01,Servicios de información,73.0
211,2022-09-01,Servicios de información,102.0
212,2022-10-01,Servicios de información,101.0
213,2022-11-01,Servicios de información,86.0


## 3. Definimos el gráfico

In [11]:

# Definición de función para generar gráfico (para la droplist)
def generate_plot(column):
    # Un gráfico por columna
    plot_data = ts_df_long[ts_df_long['variable'] == column]

    # Definimos el gráfico
    
    ## partes (lo pongo aquí porque daría error al ponerlo línea por línea por la barra)
    
    ## definimos df y variables
    ## definimos color y tamaño de línea
    ## definimos ejes
    ## tema blanco y negro
    ## eliminamos grid, definimos tamaño de figura y ponemos fondo blanco (theme-bw no va tan bien aquí como en ggplot)
    ## definimos escala de Y en términos relativos (para que los textos en annotate se vean mejor)ç
    ## definimos líneas horizontales por fecha para los períodos
    ## definimos textos, los colocamos arriba y en punto intermedio de la fecha de los períodos, además del color, la negrita y el tamaño
    
    p = ggplot(plot_data, aes(x='fecha', y='value')) + \
        geom_line(color='#3182bd', size=1.05) + \
        labs(x='Fecha', y='Número de contratos en ETTs', title=f'Evolución temporal de {column}') + \
        theme_bw() + \
        theme(panel_grid_major=element_blank(), panel_grid_minor=element_blank(),
              plot_background=element_rect(fill='white'), figure_size=(8, 4)) + \
        scale_y_continuous(limits=(0, max(plot_data['value'])*1.15), expand=(0.05, 0)) + \
        geom_vline(xintercept='2020-03-01', color='black', size=1.10) + \
        geom_vline(xintercept='2020-07-01', color='black', linetype='dotted', size=1.10) + \
        geom_vline(xintercept='2021-03-01', color='black', linetype='dotted', size=1.10) + \
        geom_vline(xintercept='2022-03-01', color='black', linetype='dotted', size=1.10) + \
        annotate('text', x='2019-09-01', y=max(plot_data['value'])*1.10, 
                 label='Pre-pandemia', size=10, color='black', ha='center', fontweight='bold') + \
        annotate('text', x='2020-05-01', y=max(plot_data['value'])*1.10, 
                 label='Lock', size=10, color='black', ha='center', fontweight='bold') + \
        annotate('text', x='2020-11-01', y=max(plot_data['value'])*1.10,
                 label='1er Año', size=10, color='black', ha='center', fontweight='bold') + \
        annotate('text', x='2021-09-01', y=max(plot_data['value'])*1.10,
                 label='2do Año', size=10, color='black', ha='center', fontweight='bold') + \
        annotate('text', x='2022-09-01', y=max(plot_data['value'])*1.10,
                 label='3er Año', size=10, color='black', ha='center', fontweight='bold')

    return p


## 4. Creamos gráfico

In [12]:
# dropdown widget con las columnas como opciones
dropdown = widgets.Dropdown(options=columns, description='Seleccione el sector')

# Definimos el output
output = widgets.Output()

# Definimos la función para cambiar gráfico según dropdown
def on_dropdown_change(change):
    output.clear_output() # limpiar output
    with output:
        # Generar gráfico de nuevo
        p = generate_plot(change.new)
        print(p)

# Unir función de limpiar dropdown 
dropdown.observe(on_dropdown_change, names='value')

# Mostrar el dropdown y el resultado
display(dropdown)
display(output)

Dropdown(description='Seleccione el sector', options=('Actividades de turismo y viajes', 'Telecomunicaciones',…

Output()

## 5. Conclusiones

1. Turismo y viajes: Se observa que durante el primer año de la pandemia la actividad se reduce completamente y la actividad normal no se recupera hasta el verano de 2021. En el verano y navidades de 2022 se observa incluso un repunte de la actividad superior al de 2019. Este comportamiento es natural debido a las características de la actividad, la cual implica una gran movilidad y, por lo tanto, un mayor riesgo de contagio y dificultad de seguimiento.


2. Telecomunicaciones: En el momento de la pandemia se observa un enorme boom, que alcanza su pico en noviembre-diciembre de 2021. En el segundo año se establiza en una media superior a la de 2019, un pico que cae considerablemente al tercer año. Este comportamiento es completamente contrario al de las actividades de turismo, lo cual obedece a las características totalmente distintas del sector: teletrabajo, aumento de la demanda de conexiones y comunicación online, ya que las restricciones obligaban a una actividad tanto de ocio como laboral de manera más telemática.


3. Ac. Postales y correos: Parece que la actividad cíclica pre-pandemia se mantiene, pero aumenta considerablemente la contratación. No obstante, después de un pico en navidades de 2022, el número de contratos disminuye, pero permanece superior a la actividad de 2019. También podemos argumentar que obedece al aumento de las comunicaciones impersonales y a distancia tanto de ciudadanos como de empresas, para evitar el riesgo de contagio y las sanciones por las restricciones.


4. Ac. de creación, artísticas y espectáculos: De la misma manera que los viajes, nos encontramos ante una actividad en donde se reúnen numerosas personas y el riesgo de contagio es alto. De ahí la forma convexa de la evolución, que no logra recuperar su actividad previa a la pandemia hasta llegar al verano de 2022.


5. Servicios de información: Observamos que, tras un pequeño declive durante la cuarentena, que parece recuperarse de manera relativamente rápida hasta alcanzar los niveles previos a la pandemia en las navidades de 2022, donde vuelve a caer, posiblemente debido a la naturaleza cíclica de la actividad y a que el requerimiento de trabajadores es menor (o se ha automatizado). 

En general, podemos diferenciar dos tendencias generales principales: 1) La pandemia ha supuesto un gran golpe a la actividad que se recupera poco a poco hasta llegar a mediados/finales de 2022, donde la actividad se recupera e incluso puede repuntar; 2) La pandemia no ha supuesto en el medio plazo un descenso de la actividad e incluso puede suponer un ligero aumento por la naturaleza de comunicación a distancia de la actividad, para después volver a la normalidad.